In [1]:
import torch
import cv2
import cvzone
from torchvision import transforms
import torchvision

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
SNet = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT").to(device)

In [4]:
SNet.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [5]:
classnames = []
with open("classes.txt", "r") as f:
    classnames = f.read().splitlines()

In [6]:
classnames

['__background__',
 'person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'N/A',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'N/A',
 'backpack',
 'umbrella',
 'N/A',
 'N/A',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'N/A',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'N/A',
 'dining table',
 'N/A',
 'N/A',
 'toilet',
 'N/A',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'N/A',
 'book',
 'clock',
 'vase',
 'scissors',
 'ted

In [7]:
image = cv2.imread("dog_man.jpeg")
image = cv2.resize(image, (640, 480))

In [8]:
image_transform = transforms.ToTensor()
type(image_transform)

torchvision.transforms.transforms.ToTensor

In [9]:
img = image_transform(image)
type(img)

torch.Tensor

In [10]:
img = img.to(device)
img

tensor([[[0.0588, 0.0314, 0.0431,  ..., 0.4353, 0.1059, 0.1843],
         [0.0784, 0.0431, 0.0471,  ..., 0.2667, 0.1647, 0.1529],
         [0.0510, 0.0314, 0.0392,  ..., 0.3333, 0.1294, 0.1294],
         ...,
         [0.7529, 0.7373, 0.7176,  ..., 0.2196, 0.3176, 0.2471],
         [0.7843, 0.7725, 0.7569,  ..., 0.1647, 0.3451, 0.3373],
         [0.6667, 0.6980, 0.7569,  ..., 0.2353, 0.3412, 0.3216]],

        [[0.1765, 0.1020, 0.0706,  ..., 0.6196, 0.2784, 0.3569],
         [0.2039, 0.1255, 0.0824,  ..., 0.4667, 0.3412, 0.3255],
         [0.1882, 0.1216, 0.0863,  ..., 0.5529, 0.3255, 0.3098],
         ...,
         [0.7882, 0.7725, 0.7529,  ..., 0.4549, 0.5412, 0.4863],
         [0.8196, 0.8078, 0.7922,  ..., 0.3882, 0.5686, 0.5686],
         [0.7098, 0.7373, 0.7961,  ..., 0.4471, 0.5608, 0.5529]],

        [[0.1412, 0.0902, 0.0667,  ..., 0.5882, 0.2314, 0.3020],
         [0.1647, 0.1098, 0.0745,  ..., 0.4078, 0.2745, 0.2627],
         [0.1451, 0.1020, 0.0784,  ..., 0.4667, 0.2471, 0.

In [11]:
with torch.inference_mode():
    pred = SNet([img])
    print(pred)
    print(pred[0].keys())

    bbox, scores, labels = pred[0]['boxes'], pred[0]['scores'], pred[0]['labels']
    bbox = bbox.cpu()
    labels = labels.cpu()
    #scores = scores.cpu()
    conf = torch.argwhere(scores > 0.7).shape[0]
    print(conf)
    for k in range(conf):
        x, y, w, h = bbox[k].numpy().astype('int')
        classname = labels[k].numpy().astype('int')
        class_detected = classnames[classname]
        cv2.rectangle(image, (x, y), (w, h), (0, 0, 255), 4)
        cvzone.putTextRect(image, class_detected, [x+8, y-12], scale=2)

cv2.imshow('frame', image)
cv2.waitKey(0)

[{'boxes': tensor([[232.1866,  76.9383, 373.4162, 458.6304],
        [423.4870, 345.9967, 518.9540, 476.0911],
        [286.0281, 153.7335, 295.1251, 221.7445],
        [289.3214, 135.8432, 327.9467, 279.6967],
        [569.5320, 243.5619, 586.2184, 276.9784],
        [225.6495, 259.5703, 237.5922, 288.5851]], device='cuda:0'), 'labels': tensor([ 1, 18, 32, 31,  1,  1], device='cuda:0'), 'scores': tensor([0.9998, 0.9985, 0.2691, 0.1274, 0.1051, 0.1020], device='cuda:0')}]
dict_keys(['boxes', 'labels', 'scores'])
2


-1